In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloaders.train import BalancedMelSampler
from dataloaders.val import MelSampler
from dataloaders.utils import csv_to_dict

from losses import NpairsLoss, MovingAverageBCE
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks
from models import NUM_FRAMES, Classifier, DeepMetricLearning

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def main(fold_idx=0, batch_size=64, saved_path="", pretrained_path="", model_name=""):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")

    train_index, val_index = get_split(fold=fold_idx)
    
    fold_train_dict = csv_to_dict(train_data.iloc[train_index])
    fold_valid_dict = csv_to_dict(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=batch_size,
        max_length=NUM_FRAMES,
        n_classes=24,
        use_cutmix=True,
        n_classes_in_batch=8,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=batch_size,
        n_classes=24,
        max_length=NUM_FRAMES,
    )

    print(' -> Preparing Model \n')
    model = get_model(
        model_name=model_name,
        saved_path=saved_path,
        pretrained_with_contrastive=False,
        pretrained_path=pretrained_path,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=50,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    step_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    ma_bce = MovingAverageBCE(
        train_data.iloc[train_index],
        start_apply_step=20 * step_per_epoch,
        momentum=0.9,
        name="moving_average_loss",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        moving_average_bce=ma_bce,
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(False, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=100,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [9]:
DEBUG = False
pretrained_folder = "../logs/2021-02-04/3/"   # simple
log_folder = "../logs/"

In [10]:
MODELS = [
    'densenet121',
#     'resnet18',
#     'resnet34',
#     'resnext50',
#     'efficientnetb2',
#     "xception",
]

BIGGER_MODELS = [  # BS = 32
#     'resnest50',
#     'efficientnetb3',
]

In [11]:
PRETRAINED_FOLDERS = {
    'densenet121': "../logs/2021-02-04/3/",
    'resnet18': "../logs/2021-02-13/14/",
    'resnet34': "../logs/2021-02-14/0/",
    'resnext50': "../logs/2021-02-14/1/",
    'efficientnetb2': "../logs/2021-02-14/2/",
    "xception": "../logs/2021-02-14/3/",
    "resnest50": "../logs/2021-02-14/5/",
    "efficientnetb3": "../logs/2021-02-14/6/",
}

In [12]:
for model_name in MODELS:
    print(f"\n\nModel {model_name}\n\n")
    if not DEBUG:
        log_folder = prepare_log_folder(LOG_PATH)
        print(f'Logging results to {log_folder}')
        create_logger(directory=log_folder, name="logs.txt")

    for fold_idx in range(5):
        tf.keras.backend.clear_session()
        print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")

        main(
            model_name=model_name,
            fold_idx=fold_idx, 
            saved_path=log_folder, 
            pretrained_path=PRETRAINED_FOLDERS[model_name] + f"pretrained_best_fold{fold_idx}.h5",
        )

        if DEBUG:
            break



Model densenet121


Logging results to ../logs/2021-02-16/1/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Loading weights from ../logs/2021-02-04/3/pretrained_best_fold0.h5

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)
 -> Training Model 

Epoch 1/100
Instructions for updating:
Use fn_output_signature instead
15/15 - 77s - loss: 0.4301 - lwlrap: 0.1761 - val_loss: 0.3393 - val_lwlrap: 0.2229
Epoch 2/100
15/15 - 7s - loss: 0.1356 - lwlrap: 0.2158 - val_loss: 0.1969 - val_lwlrap: 0.2677
Epoch 3/100
15/15 - 7s - loss: 0.1602 - lwlrap: 0.2660 - val_loss: 0.5926 - val_lwlrap: 0.2608
Epoch 4/100
15/15 - 6s - loss: 0.1880 - lwlrap: 0.3340 - val_loss: 0.1610 - val_lwlrap: 0.4544
Epoch 5/100
15/15 - 6s - loss: 0.3571 - lwlrap: 0.3489 - val_loss: 0.3296 - val_lwlrap: 0.3007
Epoch 6/100
15/15 - 6s - loss: 0.2989 - lwlrap: 0.4167 - val_loss: 0.1578 - val_lwlrap: 0.4732
Epoch 7/100
15/15 - 6s - loss: 0.2853 - lwlrap: 0.4425 - 

Epoch 79/100
15/15 - 6s - loss: 0.0986 - lwlrap: 0.8440 - val_loss: 0.1279 - val_lwlrap: 0.8024
Epoch 80/100
15/15 - 7s - loss: 0.1723 - lwlrap: 0.9241 - val_loss: 0.1262 - val_lwlrap: 0.8103
Epoch 81/100
15/15 - 6s - loss: 0.1776 - lwlrap: 0.8709 - val_loss: 0.1381 - val_lwlrap: 0.7948
Epoch 82/100
15/15 - 7s - loss: 0.1862 - lwlrap: 0.8729 - val_loss: 0.1338 - val_lwlrap: 0.8109
Epoch 83/100
15/15 - 7s - loss: 0.1911 - lwlrap: 0.8781 - val_loss: 0.1451 - val_lwlrap: 0.8112
Epoch 84/100
15/15 - 6s - loss: 0.0756 - lwlrap: 0.8714 - val_loss: 0.1495 - val_lwlrap: 0.7984
Epoch 85/100
15/15 - 7s - loss: 0.1922 - lwlrap: 0.8863 - val_loss: 0.1440 - val_lwlrap: 0.8048
Epoch 86/100
15/15 - 6s - loss: 0.0880 - lwlrap: 0.8560 - val_loss: 0.1435 - val_lwlrap: 0.7977
Epoch 87/100
15/15 - 7s - loss: 0.1755 - lwlrap: 0.8655 - val_loss: 0.1415 - val_lwlrap: 0.7863
Epoch 88/100
15/15 - 6s - loss: 0.0884 - lwlrap: 0.8508 - val_loss: 0.1343 - val_lwlrap: 0.7910
Epoch 89/100
15/15 - 6s - loss: 0.0667 -

Epoch 62/100
15/15 - 7s - loss: 0.1683 - lwlrap: 0.8397 - val_loss: 0.1184 - val_lwlrap: 0.8345
Epoch 63/100
15/15 - 6s - loss: 0.0642 - lwlrap: 0.8712 - val_loss: 0.1155 - val_lwlrap: 0.8560
Epoch 64/100
15/15 - 7s - loss: 0.0830 - lwlrap: 0.8533 - val_loss: 0.1224 - val_lwlrap: 0.8511
Epoch 65/100
15/15 - 7s - loss: 0.0656 - lwlrap: 0.8354 - val_loss: 0.1230 - val_lwlrap: 0.8518
Epoch 66/100
15/15 - 7s - loss: 0.0715 - lwlrap: 0.8710 - val_loss: 0.1240 - val_lwlrap: 0.8529
Epoch 67/100
15/15 - 7s - loss: 0.0810 - lwlrap: 0.8501 - val_loss: 0.1231 - val_lwlrap: 0.8536
Epoch 68/100
15/15 - 7s - loss: 0.1669 - lwlrap: 0.8561 - val_loss: 0.1219 - val_lwlrap: 0.8589
Epoch 69/100
15/15 - 7s - loss: 0.1797 - lwlrap: 0.8632 - val_loss: 0.1156 - val_lwlrap: 0.8597
Epoch 70/100
15/15 - 7s - loss: 0.1710 - lwlrap: 0.8659 - val_loss: 0.1183 - val_lwlrap: 0.8574
Epoch 71/100
15/15 - 7s - loss: 0.0613 - lwlrap: 0.8803 - val_loss: 0.1138 - val_lwlrap: 0.8424
Epoch 72/100
15/15 - 6s - loss: 0.0757 -

Epoch 44/100
15/15 - 7s - loss: 0.1804 - lwlrap: 0.7813 - val_loss: 0.1043 - val_lwlrap: 0.7521
Epoch 45/100
15/15 - 7s - loss: 0.1781 - lwlrap: 0.7892 - val_loss: 0.1068 - val_lwlrap: 0.7626
Epoch 46/100
15/15 - 7s - loss: 0.1177 - lwlrap: 0.7796 - val_loss: 0.1118 - val_lwlrap: 0.7651
Epoch 47/100
15/15 - 7s - loss: 0.1712 - lwlrap: 0.7696 - val_loss: 0.1129 - val_lwlrap: 0.7543
Epoch 48/100
15/15 - 6s - loss: 0.0732 - lwlrap: 0.7962 - val_loss: 0.1145 - val_lwlrap: 0.7517
Epoch 49/100
15/15 - 6s - loss: 0.1071 - lwlrap: 0.7910 - val_loss: 0.1084 - val_lwlrap: 0.7595
Epoch 50/100
15/15 - 7s - loss: 0.1169 - lwlrap: 0.8197 - val_loss: 0.1095 - val_lwlrap: 0.7538
Epoch 51/100
15/15 - 7s - loss: 0.1814 - lwlrap: 0.7945 - val_loss: 0.1168 - val_lwlrap: 0.7475
Epoch 52/100
15/15 - 6s - loss: 0.0911 - lwlrap: 0.7651 - val_loss: 0.1212 - val_lwlrap: 0.7578
Epoch 53/100
15/15 - 7s - loss: 0.1476 - lwlrap: 0.8231 - val_loss: 0.1194 - val_lwlrap: 0.7519
Epoch 54/100
15/15 - 7s - loss: 0.1644 -

Epoch 25/100
15/15 - 7s - loss: 0.0972 - lwlrap: 0.7166 - val_loss: 0.1010 - val_lwlrap: 0.8043
Epoch 26/100
15/15 - 6s - loss: 0.1664 - lwlrap: 0.7199 - val_loss: 0.0942 - val_lwlrap: 0.8056
Epoch 27/100
15/15 - 7s - loss: 0.1316 - lwlrap: 0.7022 - val_loss: 0.0934 - val_lwlrap: 0.8067
Epoch 28/100
15/15 - 7s - loss: 0.1036 - lwlrap: 0.7385 - val_loss: 0.0955 - val_lwlrap: 0.8038
Epoch 29/100
15/15 - 6s - loss: 0.1133 - lwlrap: 0.7197 - val_loss: 0.0979 - val_lwlrap: 0.8121
Epoch 30/100
15/15 - 7s - loss: 0.0985 - lwlrap: 0.7066 - val_loss: 0.0951 - val_lwlrap: 0.8075
Epoch 31/100
15/15 - 7s - loss: 0.1213 - lwlrap: 0.7460 - val_loss: 0.0966 - val_lwlrap: 0.7795
Epoch 32/100
15/15 - 7s - loss: 0.1587 - lwlrap: 0.7438 - val_loss: 0.0908 - val_lwlrap: 0.8110
Epoch 33/100
15/15 - 7s - loss: 0.1161 - lwlrap: 0.8156 - val_loss: 0.0902 - val_lwlrap: 0.8368
Epoch 34/100
15/15 - 7s - loss: 0.1366 - lwlrap: 0.7793 - val_loss: 0.0938 - val_lwlrap: 0.8327
Epoch 35/100
15/15 - 7s - loss: 0.0908 -

Epoch 6/100
15/15 - 6s - loss: 0.1797 - lwlrap: 0.3680 - val_loss: 0.1477 - val_lwlrap: 0.6327
Epoch 7/100
15/15 - 6s - loss: 0.1322 - lwlrap: 0.4056 - val_loss: 0.1397 - val_lwlrap: 0.6700
Epoch 8/100
15/15 - 6s - loss: 0.1575 - lwlrap: 0.4416 - val_loss: 0.1291 - val_lwlrap: 0.7019
Epoch 9/100
15/15 - 6s - loss: 0.1183 - lwlrap: 0.4645 - val_loss: 0.1522 - val_lwlrap: 0.6294
Epoch 10/100
15/15 - 6s - loss: 0.0981 - lwlrap: 0.5211 - val_loss: 0.1378 - val_lwlrap: 0.6782
Epoch 11/100
15/15 - 6s - loss: 0.0951 - lwlrap: 0.4867 - val_loss: 0.1432 - val_lwlrap: 0.6723
Epoch 12/100
15/15 - 6s - loss: 0.1109 - lwlrap: 0.5807 - val_loss: 0.1415 - val_lwlrap: 0.7312
Epoch 13/100
15/15 - 7s - loss: 0.2303 - lwlrap: 0.5515 - val_loss: 0.1241 - val_lwlrap: 0.7508
Epoch 14/100
15/15 - 6s - loss: 0.2496 - lwlrap: 0.6025 - val_loss: 0.1210 - val_lwlrap: 0.7699
Epoch 15/100
15/15 - 6s - loss: 0.0892 - lwlrap: 0.5732 - val_loss: 0.1147 - val_lwlrap: 0.7692
Epoch 16/100
15/15 - 6s - loss: 0.1592 - lwl

Epoch 92/100
15/15 - 7s - loss: 0.2034 - lwlrap: 0.8704 - val_loss: 0.1295 - val_lwlrap: 0.8559
Epoch 93/100
15/15 - 7s - loss: 0.0994 - lwlrap: 0.8733 - val_loss: 0.1410 - val_lwlrap: 0.8276
Epoch 94/100
15/15 - 7s - loss: 0.1940 - lwlrap: 0.8505 - val_loss: 0.1370 - val_lwlrap: 0.8284
Epoch 95/100
15/15 - 7s - loss: 0.2130 - lwlrap: 0.8608 - val_loss: 0.1359 - val_lwlrap: 0.8279
Epoch 96/100
15/15 - 6s - loss: 0.2074 - lwlrap: 0.8755 - val_loss: 0.1358 - val_lwlrap: 0.8436
Epoch 97/100
15/15 - 6s - loss: 0.1924 - lwlrap: 0.8500 - val_loss: 0.1376 - val_lwlrap: 0.8228
Epoch 98/100
15/15 - 7s - loss: 0.0767 - lwlrap: 0.8661 - val_loss: 0.1410 - val_lwlrap: 0.8342
Epoch 99/100
15/15 - 7s - loss: 0.0740 - lwlrap: 0.8692 - val_loss: 0.1583 - val_lwlrap: 0.7959
Epoch 100/100
15/15 - 7s - loss: 0.0745 - lwlrap: 0.8983 - val_loss: 0.1460 - val_lwlrap: 0.8076


In [13]:
for model_name in BIGGER_MODELS:
    print(f"\n\nModel {model_name}\n\n")
    if not DEBUG:
        log_folder = prepare_log_folder(LOG_PATH)
        print(f'Logging results to {log_folder}')
        create_logger(directory=log_folder, name="logs.txt")

    for fold_idx in range(4, 5):
        tf.keras.backend.clear_session()
        print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")

        main(
            model_name=model_name,
            batch_size=32,
            fold_idx=fold_idx, 
            saved_path=log_folder, 
            pretrained_path=PRETRAINED_FOLDERS[model_name] + f"pretrained_best_fold{fold_idx}.h5",
        )

        if DEBUG:
            break